This notebook aims to evaluate different clustering algorithms with respect to STEM algorithm

# Import necessary Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering, spectral_clustering
from sklearn.mixture import GMM
from scipy.stats import pearsonr
import scipy.cluster.hierarchy as hac
from scipy.cluster.hierarchy import dendrogram




# Load the different dataset tested
1) STEM (clustering)
2) Stirling (unclustered)

In [27]:
#stirling_df = pd.read_csv("/Users/indrikwijaya/Downloads/stirling_logdiff.txt", sep = '\t')
stirling_df = pd.read_csv("/Users/indrikwijaya/Desktop/rna_tot_mito.txt", sep = '\t', index_col = 0)
stirling_df.head()
#stirling_df.describe()


,t0,t1,t2,t3,t4,t5,t6
NM_175137,3.960592,4.095914,4.849770,4.658206,4.339691,4.389088,4.325856
NM_008044,4.315331,3.685871,4.328333,3.992102,3.750350,3.568070,3.450349
NM_207207,6.146943,6.772000,7.019586,6.848844,6.104179,5.471964,5.572717
NM_001013381,2.444733,2.903270,2.186983,1.701441,1.413871,2.650995,1.989258
NM_001013389,3.841136,4.752684,5.000866,4.826618,5.168860,4.903922,5.201029


In [4]:
stirling_stem = pd.read_csv("/Users/indrikwijaya/Downloads/stirling_stem_m60_c2.txt", sep = '\t')
stirling_stem.head()

,gene_id,Profile,time1,time2,time3,time4,time5,time6,time7
0,Q0010,55,0,1.74,-0.50,-0.03,0.61,1.07,1.65
1,Q0017,56,0,1.30,0.24,-0.40,0.89,1.40,2.08
2,Q0032,48,0,0.62,0.43,0.98,0.54,1.19,1.94
3,Q0055,48,0,0.33,0.59,0.66,0.55,1.08,1.72
4,Q0065,48,0,0.35,0.46,0.44,0.57,1.02,1.59


# Filter Significant Clusters From STEM

In [70]:
rna_cru_stem = pd.read_csv('/Users/indrikwijaya/Desktop/stem_clusters/rna_cru_m100_c3.txt', sep = '\t')
def filter_significant_clusters_stem(stem_df_file, sig_profs):
    stem_df = pd.read_csv(stem_df_file, sep = '\t')
    stem_df = stem_df.drop(['Selected', 'SPOT'], axis = 1)
    stem_df = stem_df[stem_df['Profile'].isin(sig_profs)]
    stem_df = stem_df.set_index('gene')
    stem_df.index.name = None
    sig_clust = []
    for i in sig_profs:
        cluster_i = stem_df[stem_df['Profile']==i]
        cluster_i = cluster_i[['t0', 't1','t2','t3', 't4','t5', 't6']]
        cluster_i.T.plot(legend = False, figsize=(15,7))
        plt.title('Profile %d with %d genes' % (i,len(cluster_i)), fontsize= 20)
        plt.xticks(fontsize =17)
        plt.yticks(fontsize = 17)
        #plt.show()
        cluster_i_gene = cluster_i.axes[0].tolist()
        sig_clust.append(cluster_i_gene)

    true_cluster = []
    for i in range(0,len(sig_clust)):
        true_cluster.append([i,sig_clust[i]])
    datasize = sum(len(cluster) for cluster in sig_clust)
    return true_cluster, datasize


In [114]:
sig_profs = [74,69,80,70,25,37,98,79,71,13]
rna_tot_mito_sig,datasize=filter_significant_clusters_stem('/Users/indrikwijaya/Desktop/stem_clusters/rna_tot_mito_m100_c3.txt',sig_profs)

/Users/indrikwijaya/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [ ]:
#stirling_stem = pd.read_csv("/Users/indrikwijaya/Downloads/stirling_stem_m60_c2.txt", sep = '\t')
stirling_stem = pd.read_csv("/Users/indrikwijaya/Desktop/stem_clusters/")
significant = [15,11,53,10,37,40,54,49,48,16,52,44,9,21,38,4,5,33]
stirling_stem_significant = stirling_stem[stirling_stem['Profile'].isin(significant)]
significant_clusters = []
for i in significant:
    cluster_i = stirling_stem_significant[stirling_stem_significant['Profile']==i]
    cluster_i = cluster_i.set_index('gene_id')
    cluster_i.index.name = None
    cluster_i = cluster_i[['time1','time2', 'time3', 'time4', 'time5', 'time6', 'time7']]
#     cluster_i.T.plot(legend = False, figsize=(20,10))
#     plt.title ('%d genes' % len(cluster_i), fontsize = 45)
#     plt.xticks(fontsize=35)
#     plt.yticks(fontsize=35)
    #plt.savefig('/Users/indrikwijaya/Desktop/stirling_stem_significant/Cluster_%d_%d.png' % (i,len(cluster_i)))
    #plt.show()
    
    cluster_i_gene = list(cluster_i.iloc[:,0])
    significant_clusters.append(cluster_i_gene)

#print(significant_clusters[0])
datasize = sum(len(cluster) for cluster in significant_clusters)
datasize

In [8]:
true_cluster = []
for i in range(0,len(significant_clusters)):
    true_cluster.append([i,significant_clusters[i]])
#true_cluster
        


# K-means Clustering

In [90]:
kmeans = KMeans(n_clusters = 10, random_state=1).fit(stirling_df.values)
kmeans_labels = kmeans.predict(stirling_df.values)

s = pd.Series(kmeans_labels)
clusters = s.unique()
kmeans_cluster = []
for i in range(0, len(clusters)):
    cluster_indices = s[s==clusters[i]].index
    #print("Cluster %d number of entries %d" % (c, len(cluster_indices)))
    
    gene_cluster = stirling_df.T.iloc[:,cluster_indices].transpose().index
    kmeans_cluster.append([i, list(gene_cluster)])
    
#     stirling_df.T.iloc[:, cluster_indices].plot(legend = False, figsize=(20,10))
#     plt.title('%d genes' % len(gene_cluster), fontsize = 45)
#     plt.xticks(fontsize=35)
#     plt.yticks(fontsize = 35)
    #plt.savefig("/Users/indrikwijaya/Desktop/kmeans_cluster_18/Cluster_%d_%d" % (clusters[i], len(gene_cluster)))
    #plt.show()


# GMM Clustering

In [73]:
gmm = GMM(n_components = 10,covariance_type = 'full').fit(stirling_df.values)
gmm_labels = gmm.predict(stirling_df.values)

s = pd.Series(gmm_labels)
clusters = s.unique()
gmm_cluster = []
for i in range(0, len(clusters)):
    cluster_indices = s[s==clusters[i]].index
    #print("Cluster %d number of entries %d" % (c, len(cluster_indices)))
    gene_cluster = stirling_df.T.iloc[:,cluster_indices].transpose().index
    gmm_cluster.append([i, list(gene_cluster)])
    
#     stirling_df.T.iloc[:, cluster_indices].plot(legend = False, figsize=(20,10))
#     plt.title('%d genes' % len(gene_cluster), fontsize = 45)
#     plt.xticks(fontsize=35)
#     plt.yticks(fontsize = 35)
    #plt.savefig("/Users/indrikwijaya/Desktop/gmm_cluster_bic_plots_18/Cluster_%d_%d" % (clusters[i], len(gene_cluster)))
    #plt.show()


/Users/indrikwijaya/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:52: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/indrikwijaya/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The functon distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/indrikwijaya/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/indrikwijaya/anaconda3/lib/python3.6/site-packag

# Hierachical Clustering

In [74]:
def pearson_affinity(M):
   return 1 - np.array([[pearsonr(a,b)[0] for a in M] for b in M])


In [75]:
#Fit into ward hierarchical clustering
hac_ward = AgglomerativeClustering(n_clusters=10, 
                                      linkage = 'ward',
                                      affinity = 'euclidean')
hac_ward = hac_ward.fit(stirling_df.values)

s = pd.Series(hac_ward.labels_)
clusters = s.unique()
hac_ward_cluster = []
for c in clusters:
    cluster_indices = s[s==c].index
    #print("Cluster %d number of entries %d" % (c, len(cluster_indices)))
    stirling_df.T.iloc[:, cluster_indices].plot(legend = False)
    gene_cluster = stirling_df.T.iloc[:,cluster_indices].transpose().index
    hac_ward_cluster.append([c, list(gene_cluster)])

NameError: name 'stirling_hac_ward' is not defined

In [14]:
#Plot the different clusters
for i in range(0, len(clusters)):
    cluster_indices = s[s==clusters[i]].index
    #print("Cluster %d number of entries %d" % (c, len(cluster_indices)))
    
    stirling_df.T.iloc[:, cluster_indices].plot(legend = False, figsize=(20,10))
    plt.title('%d genes' % len(gene_cluster), fontsize = 45)
    plt.xticks(fontsize=35)
    plt.yticks(fontsize = 35)
    #plt.savefig("/Users/indrikwijaya/Desktop/hac_ward_18/Cluster_%d_%d" % (clusters[i], len(gene_cluster)))
    #plt.show()

/Users/indrikwijaya/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [ ]:
#Fit into complete hierarchical clustering
hac_complete = AgglomerativeClustering(n_clusters=10, 
                                      linkage = 'complete',
                                      affinity = pearson_affinity)
hac_complete = hac_complete.fit(stirling_df.values)

s = pd.Series(hac_complete.labels_)
clusters = s.unique()
hac_complete_cluster = []
for c in clusters:
    cluster_indices = s[s==c].index
    #print("Cluster %d number of entries %d" % (c, len(cluster_indices)))
    #stirling_df.T.iloc[:, cluster_indices].plot(legend = False)
    gene_cluster = stirling_df.T.iloc[:,cluster_indices].transpose().index
    hac_complete_cluster.append([c, list(gene_cluster)])

In [ ]:
#Plot the different clusters
for i in range(0, len(clusters)):
    cluster_indices = s[s==clusters[i]].index
    #print("Cluster %d number of entries %d" % (c, len(cluster_indices)))
    gene_cluster = stirling_df.T.iloc[:,cluster_indices].transpose().index
    hac_complete_cluster.append([i, list(gene_cluster)])
    
    stirling_df.T.iloc[:, cluster_indices].plot(legend = False, figsize=(20,10))
    
    plt.title('%d genes' % len(gene_cluster), fontsize = 45)
    plt.xticks(fontsize=35)
    plt.yticks(fontsize = 35)
    #plt.savefig("/Users/indrikwijaya/Desktop/hac_complete_18/Cluster_%d_%d" % (clusters[i], len(gene_cluster)))
    #plt.show()

In [76]:
#Fit into average hierarchical clustering
hac_avg = AgglomerativeClustering(n_clusters=10, 
                                      linkage = 'average',
                                      affinity = pearson_affinity)
hac_avg = hac_avg.fit(stirling_df.values)

s = pd.Series(hac_avg.labels_)
clusters = s.unique()
hac_avg_cluster = []
for c in clusters:
    cluster_indices = s[s==c].index
    #print("Cluster %d number of entries %d" % (c, len(cluster_indices)))
    stirling_df.T.iloc[:, cluster_indices].plot(legend = False)
    gene_cluster = stirling_df.T.iloc[:,cluster_indices].transpose().index
    hac_avg_cluster.append([c, list(gene_cluster)])

In [190]:
#Plot the different clusters
for i in range(0, len(clusters)):
    cluster_indices = s[s==clusters[i]].index
    #print("Cluster %d number of entries %d" % (c, len(cluster_indices)))
    gene_cluster = stirling_df.T.iloc[:,cluster_indices].transpose().index
    hac_avg_cluster.append([i, list(gene_cluster)])
    stirling_df.T.iloc[:, cluster_indices].plot(legend = False, figsize=(20,10))
    plt.title('%d genes' % len(gene_cluster), fontsize = 45)
    plt.xticks(fontsize=35)
    plt.yticks(fontsize = 35)
    #plt.savefig("/Users/indrikwijaya/Desktop/hac_avg_18/Cluster_%d_%d" % (clusters[i], len(gene_cluster)))
    #plt.show()

/Users/indrikwijaya/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


# Plot Dendrograms

In [15]:
title_size = 85
figsize =(85,65)
xlabel_size = 85
ylabel_size = 85
xticks_size = 85
yticks_size = 85
def fancy_dendrogram(model,*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)
    
    
    #children of hierarchical clustering
    children = model.children_
    
    #distances btw each pair of children
    #since we don't have this information, we can use a uniform one for plotting
    distance = np.arange(children.shape[0])
    
    #The no of observations contained in each cluster level
    no_of_observations = np.arange(2, children.shape[0]+2)
    
    #create linkage matrix and then plot the dendrogram
    linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)
    ddata = hac.dendrogram(linkage_matrix,*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram', fontsize= title_size)
        plt.xlabel('sample index or (cluster size)', fontsize = xlabel_size)
        plt.ylabel('distance', fontsize = ylabel_size)
        plt.xticks(fontsize=xticks_size, rotation =90)
        plt.yticks(fontsize=yticks_size, rotation =90)
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata
    


In [39]:
Z = stirling_hac_complete
plt.figure(figsize=figsize)
plt.title("hierarchical clustering dendogram")
plt.xlabel('sample index', fontsize = xlabel_size)
plt.ylabel('distance', fontsize = ylabel_size)
plt.yticks(fontsize = yticks_size, rotation = 90)
fancy_dendrogram(
    Z,
    leaf_rotation = 90., #rotate the x axis labels
    leaf_font_size= 45., #font size for the x axis labels
    max_d = 0.02
)
plt.show()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [38]:
Z = stirling_hac_avg
plt.figure(figsize=figsize)
plt.title("hierarchical clustering dendogram")
plt.xlabel('sample index', fontsize = xlabel_size)
plt.ylabel('distance', fontsize = ylabel_size)
plt.yticks(fontsize = yticks_size, rotation = 90)
fancy_dendrogram(
    Z,
    leaf_rotation = 90., #rotate the x axis labels
    leaf_font_size= 45., #font size for the x axis labels
    max_d = 0.02
)
plt.show()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


# Compare the performances of different clustering algorithm vs STEM

In [14]:
datasize = sum(len(cluster) for cluster in significant_clusters)


def factorial(n):
    if n == 1:
        return 1
    else:
        return n*factorial(n-1)
    
def intersect(a,b):
    return list(set(a)&set(b))

def label_to_true(true_cluster,test_cluster):
    for i in range(0, len(test_cluster)):
        max_label = 0
        for j in range(0, len(true_cluster)):
            max_intersect = len(intersect(test_cluster[i][1],true_cluster[j][1]))
            if len(intersect(test_cluster[i][1],true_cluster[j][1]))>max_label:
                max_label = max_intersect
                test_cluster[i][0] = true_cluster[j][0]
    return test_cluster

                   
def clustering_accuracy(true_cluster, test_cluster):
    count = 0
    for i in test_cluster:
        for j in true_cluster:
            if i[0] == j[0]:
                count += len(intersect(i[1],j[1]))
                #print(count)
    return count/datasize

In [69]:
clustering_accuracy(rna_tot_sig,label_to_true(rna_tot_sig,kmeans_cluster))

0.36890243902439024

In [77]:
clustering_accuracy(true_cluster,label_to_true(true_cluster,gmm_cluster))

0.0

In [78]:
clustering_accuracy(true_cluster, label_to_true(true_cluster, kmeans_cluster))

0.0

In [79]:
clustering_accuracy(true_cluster, label_to_true(true_cluster,hac_ward_cluster))

NameError: name 'hac_ward_cluster' is not defined

In [ ]:
clustering_accuracy(true_cluster, label_to_true(true_cluster,hac_complete_cluster))

In [ ]:
clustering_accuracy(true_cluster, label_to_true(true_cluster,hac_avg_cluster))

In [82]:
a = np.empty((datasize,18))

In [151]:
def indicator_matrix(cluster):

    cluster_sizes = [len(i[1]) for i in cluster]
    confusion_mat = np.empty((datasize,1))
    index = 0
    for k in range(0, len(cluster_sizes)):
        #print(cluster_sizes[k])
        cluster_k = np.zeros((datasize,1))
        #cluster_k[:index+cluster_sizes[k]] = 1/cluster_sizes[k]
        cluster_k[index:index+cluster_sizes[k]] = 1
        index += cluster_sizes[k]
        #print(cluster_k)
        confusion_mat = np.concatenate((confusion_mat,cluster_k),axis = 1)

    confusion_mat = confusion_mat[:,1:]
    
    return confusion_mat


In [ ]:
true_indicator = indicator_matrix(true_cluster)
gmm_indicator = indicator_matrix(label_to_true(true_cluster,gmm_cluster))
confusion_gmm = np.dot(true_indicator.T, gmm_indicator)
confusion_gmm

In [54]:
""" Global variables:
       n = number of vertices on each side
       U,V vertex sets
       lu,lv are the labels of U and V resp.
       the matching is encoded as 
       - a mapping Mu from U to V, 
       - and Mv from V to U.
    
    The algorithm repeatedly builds an alternating tree, rooted in a
    free vertex u0. S is the set of vertices in U covered by the tree.
    For every vertex v, T[v] is the parent in the tree and Mv[v] the
    child.
    The algorithm maintains minSlack, s.t. for every vertex v not in
    T, minSlack[v]=(val,u1), where val is the minimum slack
    lu[u]+lv[v]-w[u][v] over u in S, and u1 is the vertex that
    realizes this minimum.
    Complexity is O(n^3), because there are n iterations in
    maxWeightMatching, and each call to augment costs O(n^2). This is
    because augment() makes at most n iterations itself, and each
    updating of minSlack costs O(n).
    """

def improveLabels(val):
    """ change the labels, and maintain minSlack. 
    """
    for u in S:
        lu[u] -= val
    for v in V:
        if v in T:
            lv[v] += val
        else:
            minSlack[v][0] -= val

def improveMatching(v):
    """ apply the alternating path from v to the root in the tree. 
    """
    u = T[v]
    if u in Mu:
        improveMatching(Mu[u])
    Mu[u] = v
    Mv[v] = u

def slack(u,v): return lu[u]+lv[v]-w[u][v]

def augment():
    """ augment the matching, possibly improving the lablels on the way.
    """
    while True:
        # select edge (u,v) with u in S, v not in T and min slack
        ((val, u), v) = min([(minSlack[v], v) for v in V if v not in T])
        assert u in S
        if val>0:        
            improveLabels(val)
        # now we are sure that (u,v) is saturated
        assert slack(u,v)==0
        T[v] = u                            # add (u,v) to the tree
        if v in Mv:
            u1 = Mv[v]                      # matched edge, 
            assert not u1 in S
            S[u1] = True                    # ... add endpoint to tree 
            for v in V:                     # maintain minSlack
                if not v in T and minSlack[v][0] > slack(u1,v):
                    minSlack[v] = [slack(u1,v), u1]
        else:
            improveMatching(v)              # v is a free vertex
            return

def maxWeightMatching(weights):
    """ given w, the weight matrix of a complete bipartite graph,
        returns the mappings Mu : U->V ,Mv : V->U encoding the matching
        as well as the value of it.
    """
    global U,V,S,T,Mu,Mv,lu,lv, minSlack, w
    w  = weights
    n  = len(w)
    U  = V = range(n)
    lu = [ max([w[u][v] for v in V]) for u in U]  # start with trivial labels
    lv = [ 0                         for v in V]
    Mu = {}                                       # start with empty matching
    Mv = {}
    while len(Mu)<n:
        free = [u for u in V if u not in Mu]      # choose free vertex u0
        u0 = free[0]
        S = {u0: True}                            # grow tree from u0 on
        T = {}
        minSlack = [[slack(u0,v), u0] for v in V]
        augment()
    #                                    val. of matching is total edge weight
    val = sum(lu)+sum(lv)
    return (Mu, Mv, val)


  
#  a small example 

print(maxWeightMatching([[1,2,3,4],[2,4,6,8],[3,6,9,12],[4,8,12,16]]))

# read from standard input a line with n
# then n*n lines with u,v,w[u][v]

# n = int(raw_input())
# w = [[0 for v in range(n)] for u in range(n)]
# for _ in range(n*n):
#     u,v,w[u][v] = map(int, raw_input().split())

# print(maxWeightMatching(w))

({0: 0, 1: 1, 2: 2, 3: 3}, {3: 3, 2: 2, 1: 1, 0: 0}, 30)


In [99]:
rna_tot_sig[0]

[0,
 ['NM_175137',
  'NM_009941',
  'NM_173180',
  'NM_172582',
  'NM_026452',
  'NM_025909',
  'NM_029272',
  'NM_007505',
  'NM_001252555',
  'NM_030087',
  'NM_025597',
  'NM_009775',
  'NM_175438',
  'NM_008993',
  'NM_181040',
  'NM_019826',
  'NM_175094',
  'NM_001285783',
  'NM_001033136',
  'NM_016772',
  'NM_011694',
  'NM_019694',
  'NM_026688',
  'NM_008133',
  'NM_178848',
  'NM_153150',
  'NM_001033305',
  'NM_021534',
  'NM_024197',
  'NM_024478',
  'NM_145999',
  'NM_080633',
  'NM_177041',
  'NM_030717',
  'NM_153176',
  'NM_025927',
  'NM_030225',
  'NM_025351',
  'NM_001017429',
  'NM_008618',
  'NM_025862',
  'NM_025641',
  'NM_017366',
  'NM_172687',
  'NM_008837',
  'NM_025313',
  'NM_172644',
  'NM_019875',
  'NM_145614',
  'NM_013885',
  'NM_133666',
  'NM_138670',
  'NM_008212',
  'NM_145494',
  'NM_025827',
  'NM_025352',
  'NM_025358',
  'NM_133783',
  'NM_025689',
  'NM_024220',
  'NM_012029',
  'NM_026172',
  'NM_001037999',
  'NM_024179',
  'NM_173866',
  '

In [115]:
def generate_clusters(time_series_df, cluster_model,cluster_type):

    if cluster_type[:3] == 'HAC':
        cluster_labels = cluster_model.fit(time_series_df.values).labels_
    else:
        cluster_labels = cluster_model.fit(time_series_df.values).predict(time_series_df.values)

    s = pd.Series(cluster_labels)
    clusters = s.unique()
    clusters_id = []

    for i in range(0, len(clusters)):
        cluster_indices = s[s==clusters[i]].index
        gene_cluster = time_series_df.T.iloc[:,cluster_indices].transpose().index
        # gene_file = open('/Users/indrikwijaya/Desktop/stem_compare/'+cluster_type+'/cluster_'+str(i)+'.txt', 'w')
        # gene_file.write("\n".join(list(gene_cluster.values)))
        clusters_id.append([i, list(gene_cluster)])

        # time_series_df.T.iloc[:, cluster_indices].plot(legend = False,figsize = (20,10))
        # plt.title('%d genes' % len(gene_cluster), fontsize = 45)
        # plt.xticks(fontsize = 35)
        # plt.yticks(fontsize = 35)
        # plt.savefig("%s_Cluster_%d_%d" % (cluster_type,clusters[i], len(gene_cluster)))

    return clusters_id

In [116]:
time_series_df = pd.read_csv('/Users/indrikwijaya/Desktop/rna_logrpkm/rna_tot_mito_diff0.txt',sep ='\t')
df_kmeans = KMeans(n_clusters=len(rna_tot_sig),random_state =1)
test = generate_clusters(time_series_df, df_kmeans, 'KMeans') 

In [118]:
weights = {}
for j in rna_tot_mito_sig:
    j_weight = {}
    for i in test:
        j_weight[i[0]] = len(intersect(j[1],i[1]))
    weights[j[0]]=j_weight
weights        

{0: {0: 42, 1: 0, 2: 4, 3: 66, 4: 25, 5: 12, 6: 1, 7: 7, 8: 0, 9: 0},
 1: {0: 0, 1: 0, 2: 0, 3: 41, 4: 67, 5: 21, 6: 11, 7: 15, 8: 0, 9: 0},
 2: {0: 49, 1: 0, 2: 12, 3: 10, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0},
 3: {0: 36, 1: 0, 2: 6, 3: 2, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0},
 4: {0: 0, 1: 20, 2: 5, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 32, 9: 4},
 5: {0: 2, 1: 12, 2: 43, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0},
 6: {0: 3, 1: 0, 2: 0, 3: 20, 4: 11, 5: 7, 6: 1, 7: 0, 8: 0, 9: 0},
 7: {0: 2, 1: 4, 2: 23, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0},
 8: {0: 12, 1: 0, 2: 8, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0},
 9: {0: 0, 1: 14, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 5, 9: 0}}

In [119]:
w=[]
for i in weights:
    w_i=weights[i]
    weight=[]
    for j in w_i:
        weight.append(w_i[j])
    w.append(weight)

In [120]:
maxWeightMatching(w)

({0: 3, 1: 4, 2: 0, 3: 6, 4: 8, 5: 2, 6: 5, 7: 9, 8: 7, 9: 1},
 {0: 2, 1: 9, 2: 5, 3: 0, 4: 1, 5: 6, 6: 3, 7: 8, 8: 4, 9: 7},
 278)

In [87]:
type(maxWeightMatching(w)[2])

int